In [1]:
import os
import pandas as pd
import numpy as np
import json
from openai import AzureOpenAI, OpenAI
from modules import *
from tqdm import tqdm
from utils import query_gpt_openai, parse_and_select, read_config, login_gis_portal
from utils import get_prompt_for_extracting_modules

%load_ext autoreload
%autoreload 2

2025-11-27 11:58:51.263218: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Get templates to extract module lists

In [ ]:
ask_num = 10
subscription_key =''
endpoint =''
# model_name = "Llama-3.3-70B-Instruct"
# deployment = "Llama-3.3-70B-Instruct"
# model_name = "Llama-4-Scout-17B-16E-Instruct"
# deployment = "Llama-4-Scout-17B-16E-Instruct"
# model_name = "gpt-oss-120b"
# deployment = "gpt-oss-120b"
# model_name = "Meta-Llama-3.1-8B-Instruct"
# deployment = "Meta-Llama-3.1-8B-Instruct"
model_name = "o4-mini"
deployment = "o4-mini"
# model_name = "Qwen/QwQ-32B"
# deployment = "Qwen/QwQ-32B"
# model_name = "grok-3-mini"
# deployment = "grok-3-mini"

api_version = "2025-04-01-preview"
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)



In [ ]:
model_name = "gpt-5.1"
deployment = "gpt-5.1"
client = OpenAI(
        # base_url="https://router.huggingface.co/v1",
        api_key=''
    )

In [3]:
target_ccode = 'KOR'
country_name = 'Korea'

target_var = 'GRDP'
target_var_desc = 'a regional GDP'
# target_var_desc = 'a regional manufacturing GDP'
# target_var = 'population'
# target_var_desc = 'the population'
#target_var = 'age_working_population'
#target_var_desc = 'the working-age population'
#target_var = 'age_old_dependency_ratio'
#target_var_desc = 'the old-age dependency ratio'
#target_var = 'age_old'
#target_var_desc = 'the population of elderly people (aged 60 and older)'
#target_var = 'education_high'
#target_var_desc = 'the proportion of highly-educated individuals (at least university graduates)'
#target_var = 'education_secondary'
#target_var_desc = 'the proportion of secondary school graduates'
#target_var = 'education_primary'
#target_var_desc = 'the proportion of people with minimal educational attainment'
#target_var = 'unemployment_rate'
#target_var_desc = 'the unemployment rate'
# target_var = 'LPR'
# target_var_desc = 'the labour force participation rate' 
#target_var = 'literacy_rate'
#target_var_desc = 'the literacy rate' 
#target_var = 'under_five_mortality'
#target_var_desc = 'the mortality rate of children under the age of 5'
#target_var = 'infant_mortality'
#target_var_desc = 'the infant mortality rate'

example_question = f'Which information is useful to infer {target_var_desc} of {country_name}?'
target_question = f"Infer {target_var_desc} as a score within [0, 1] from given location's description. Answer the score only."
fewshot_target_question = f"Infer {target_var_desc} from given location's description. Answer the numeric score only."

print(target_var)
print('example_question :', example_question)
print('target_question :', target_question)
print('fewshot_target_question :', fewshot_target_question)

GRDP
example_question : Which information is useful to infer a regional GDP of Korea?
target_question : Infer a regional GDP as a score within [0, 1] from given location's description. Answer the score only.
fewshot_target_question : Infer a regional GDP from given location's description. Answer the numeric score only.


In [5]:
prompt_version ='2'
# prompt_version ='2_NK'
prompt = get_prompt_for_extracting_modules(example_question, version=prompt_version)
prompts = [prompt for _ in range(ask_num)]
print(prompts[0])

Given a modular set, determine the sequence of modules that should be executed with inputs to solve the below question.

The modules are defined as follows:
- get_address(Loc): Get address of given location.

- get_area(Loc): Get area size of given location's district.

- get_night_light(Loc): Get nightlight intensity of given location.

- get_distance_to_nearest_target(Loc, Class): Get distance of given location to class. Class should be one of the element in ['port']

- get_landuse_sum(Loc, Class): Get the number of satelite images of class within the total area. Class should be one of the element in ["Residential", "Agricultural", "Commercial", "Factory", "Mining", "Power station", "Transportation", "Sports", "Religious", "Water", "Others"].
    - "Residential": Primarily used for housing, including dense and less dense urban areas.
    - "Agricultural": Dedicated to growing crops or raising livestock, ranging from irrigated lands to mixed cultivation fields.
    - "Commercial": Cen

In [6]:
example_question

'Which information is useful to infer a regional GDP of Korea?'

In [8]:
answers = query_gpt_openai(prompts, client, model=deployment, temperature=0.5, max_completion_tokens=20000, max_try_num=10)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [02:24<00:00, 14.46s/it]


In [9]:
answers

['---\nThought: What is one of the most important modules I should call to answer this question?  \nAction: Night‐time light intensity is widely used as a proxy for economic activity and GDP. Therefore I\'ll call get_night_light(Loc)\n\nThought: Then, what should I additionally call to normalize and compare different regions?  \nAction: I need the size of the region to normalize all proxies by area. Therefore I\'ll call get_area(Loc)\n\nThought: What next proxy relates directly to the service sector contribution?  \nAction: The extent of commercial land use indicates service and trade activity. Therefore I\'ll call get_landuse_sum(Loc, "Commercial")\n\nThought: What module captures manufacturing capacity and industrial output?  \nAction: Factory land cover correlates with industrial GDP. Therefore I\'ll call get_landuse_sum(Loc, "Factory")\n\nThought: Which module shows transport infrastructure that supports trade and logistics?  \nAction: Transportation land use (roads, rail, ports) r

In [39]:
# with open(f'selected_modules/llms/{target_ccode}_{model_name}.json','w') as f:
#     json.dump({'s_modules':answers},f,indent=4)

In [13]:
import re
def to_double_quoted(s):
    # 문자열이 큰따옴표나 작은따옴표로 둘러싸여 있으면 제거
    if (s.startswith('"') and s.endswith('"')) or (s.startswith("'") and s.endswith("'")):
        s = s[1:-1]
    # 내부의 큰따옴표를 작은따옴표로 변환
    s = s.replace('"', "'").strip()
    # 문자열을 큰따옴표로 감싸서 반환
    # print(s)
    return f"{s}"
ans_dict ={}
for i in answers:
    if i == -1:
        i = ""
    modules = list(dict.fromkeys(re.findall(r'^\d.*\n', i, re.MULTILINE)))
    # print(modules)
    for module in modules:
        
        temp = module[3:].replace('`',"").replace('\n','')
        if temp in ans_dict:
            ans_dict[temp] +=1
        else:
            ans_dict[temp] =1

s_modules = np.array([*ans_dict.keys()])[np.array([*ans_dict.values()]) >= len(answers) // 4]
# print(s_modules)
converted = [to_double_quoted(s) for s in s_modules]

# 중복 제거를 위해 dict를 이용 (순서 유지)
s_modules = list(dict.fromkeys(converted))
print(s_modules)

['get_area(Loc)', 'get_poi_info(Loc)', 'get_night_light(Loc)', "get_landuse_sum(Loc, 'Residential')", "get_landuse_sum(Loc, 'Commercial')", "get_landuse_sum(Loc, 'Factory')", "get_landuse_sum(Loc, 'Agricultural')", "get_distance_to_nearest_target(Loc, 'port')", 'get_air_info(Loc)', 'get_air_info([Loc])', 'get_night_light([Loc])', 'get_poi_info([Loc])', "get_landuse_sum([Loc], 'Commercial')", "get_landuse_sum([Loc], 'Factory')", "get_landuse_sum([Loc], 'Transportation')", "get_landuse_sum([Loc], 'Power station')", 'get_area([Loc])', "get_landuse_sum([Loc], 'Residential')", "get_distance_to_nearest_target([Loc], 'port')", "get_landuse_sum(Loc, 'Transportation')", "get_landuse_sum(Loc, 'Power station')"]


In [15]:
s_modules

['get_area(Loc)',
 'get_poi_info(Loc)',
 'get_night_light(Loc)',
 "get_landuse_sum(Loc, 'Residential')",
 "get_landuse_sum(Loc, 'Commercial')",
 "get_landuse_sum(Loc, 'Factory')",
 "get_landuse_sum(Loc, 'Agricultural')",
 "get_distance_to_nearest_target(Loc, 'port')",
 'get_air_info(Loc)',
 'get_air_info([Loc])',
 'get_night_light([Loc])',
 'get_poi_info([Loc])',
 "get_landuse_sum([Loc], 'Commercial')",
 "get_landuse_sum([Loc], 'Factory')",
 "get_landuse_sum([Loc], 'Transportation')",
 "get_landuse_sum([Loc], 'Power station')",
 'get_area([Loc])',
 "get_landuse_sum([Loc], 'Residential')",
 "get_distance_to_nearest_target([Loc], 'port')",
 "get_landuse_sum(Loc, 'Transportation')",
 "get_landuse_sum(Loc, 'Power station')"]

In [16]:
if 'get_address(Loc)' not in s_modules:
    s_modules = np.insert(s_modules, 0, 'get_address(Loc)')
elif np.where(s_modules == 'get_address(Loc)')!=0 :
    address_index = np.where(s_modules == 'get_address(Loc)')[0]
    # address_index = np.where(s_modules == 'get_address(Loc)')[0][0]
    s_modules = np.delete(s_modules, address_index)
    s_modules = np.insert(s_modules, 0, 'get_address(Loc)')

In [17]:
s_modules = [elem.replace('Func=','').replace('Class=','class=').replace('_target(Loc, class=','_target(Loc, target_name=').replace('class=','target_class=').replace("\"","'").replace("get_landcover_ratio(Loc, '","get_landcover_ratio(Loc, target_class='").replace("get_distance_to_nearest_target(Loc, '","get_distance_to_nearest_target(Loc, target_name='").replace('Loc=x', 'loc=x') for elem in s_modules]
s_modules

['get_address(Loc)',
 'get_area(Loc)',
 'get_poi_info(Loc)',
 'get_night_light(Loc)',
 "get_landuse_sum(Loc, 'Residential')",
 "get_landuse_sum(Loc, 'Commercial')",
 "get_landuse_sum(Loc, 'Factory')",
 "get_landuse_sum(Loc, 'Agricultural')",
 "get_distance_to_nearest_target(Loc, target_name='port')",
 'get_air_info(Loc)',
 'get_air_info([Loc])',
 'get_night_light([Loc])',
 'get_poi_info([Loc])',
 "get_landuse_sum([Loc], 'Commercial')",
 "get_landuse_sum([Loc], 'Factory')",
 "get_landuse_sum([Loc], 'Transportation')",
 "get_landuse_sum([Loc], 'Power station')",
 'get_area([Loc])',
 "get_landuse_sum([Loc], 'Residential')",
 "get_distance_to_nearest_target([Loc], 'port')",
 "get_landuse_sum(Loc, 'Transportation')",
 "get_landuse_sum(Loc, 'Power station')"]

In [18]:
# with open(f'selected_modules/llms_poi_air/{target_ccode}_{target_var}_o4-mini_seg.json','w') as f:
with open(f'selected_modules/llms_poi_air/{target_ccode}_{target_var}_{model_name}.json','w') as f:
    json.dump({'s_modules':list(s_modules)},f,indent=4)

In [13]:
f'selected_modules/llms_poi_air/{target_ccode}_{target_var}_QwQ.json'

'selected_modules/llms_poi_air/KOR_GRDP_QwQ.json'